In [ ]:
import os
import glob
from tqdm import tqdm

import numpy as np
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors

from astropy.io import fits

os.environ["PYSYN_CDBS"]='/path/to/your/pysynphot_data/grp/redcat/trds'
import pysynphot as S

from Cthulhu.misc import read_cross_section_file
from Cthulhu.core import summon, compute_cross_section
from Cthulhu.VALD import process_VALD_file

The following three cells are what I used to generate my 1 nbar, 1000 K cross sections for the atomic species of interest in our work. They take quite a while to run and require a bit of set-up to make happen. In particular, the atomic cross sections require that you get a VALD account and request all the relevant datasets first. I have gone ahead and saved the results of all of this rigmarole out into cross-sections.npy and cross-sections-bin.npy, which I have supplied alongside this notebook. Feel free to skip the next three cells and simply load the numpy dictionaries that I have provided, unless you want to change the cross section data at all, in which case more power to you.

Due to the size of the full cross-section dict (~600 Mb), I cannot upload that file to GitHub. So if you want to remake the plot of GJ 367's spectrum, you will have to produce the cross-sections yourself. That is unfortunate. You can still make every other plot in the paper though, the notebook will just throw you some notices when a routine is getting skipped because it did not have access to the full cross-section dictionary.

In [ ]:
### GETTING ATOMIC CROSS-SECTIONS FROM THE OLD ONE ###
atoms = ('S','C','O','Na','K')
database = 'VALD'
for atom in tqdm(atoms, desc='Computing atomic cross-sections with Cthulhu...'):
    nu_min = 1000                   # 1/cm
    nu_max = 100000                 # 1/cm
    process_VALD_file(species = atom, ionization_state = 1, VALD_data_dir = './VALD_Line_Lists/')
    summon(database=database, species = atom, VALD_data_dir='./VALD_Line_Lists/', ionization_state = 1)

    P = 1e-9                        # Pressure in bars
    T = 1000.0                      # Temperature in Kelvin
    input_directory = './input/'    # Top level directory containing line lists

    compute_cross_section(database=database, species = atom, pressure = P,
                            temperature = T, input_dir = input_directory,
                            ionization_state = state, nu_out_min=nu_min, nu_out_max=nu_max,
                            verbose=True, N_cores=7)

In [ ]:
### GETTING THE CROSS SECTIONS TO USE FOR THE REST OF THIS NOTEBOOK ###
absorption_dict = {}
full_cross_dict = {}

P = 1e-9
T = 1000


atoms = ('C','O','Na','S','K')
masses = (12.011,15.999,22.990,32.065,39.098)
database = 'VALD'
for k in tqdm(range(len(atoms)),desc='Getting atomic cross sections...'):
    atom = atoms[k]
    mass = masses[k]
    state = 1
    state_str = '_I'
    
    # These read out in 1/cm, cm^2
    nu, sigma = read_cross_section_file(species=atom,database=database,ionization_state=state,
                                        filename=atom+state_str+"_T"+"{:.1F}".format(T)+"K_log_P"+str(round(np.log10(P),1))+"_H2-He_sigma.txt")

    # Convert wavenumbers in 1/cm to wavelengths in micron
    waves = np.asarray([10*1000/i for i in nu]) # in um

    full_cross_dict[atom+state_str] = [waves,sigma,mass]

    # Take the strongest line available on the 0.1-10 um range,
    # but bin it down to comparable to resolution elements.
    max_index = np.argmax(sigma)
    if atom+state_str == 'S_I':
        # Hardcoding to pull on the 180.7311 nm line instead, since it is NIST-verified
        max_index = (np.abs(waves - 0.1807311)).argmin()
    max_wave = waves[max_index]

    # For the C, O, S lines, I'm opting for a bin size of 0.1 AA.
    # The COS gratings and STIS Echelles can reach dispersions of 0.01 AA per pixel,
    # but I want to bin several resolution elements to increase SNR.
    lower_wave = max_wave - 0.5e-5 # - 0.05 AA
    upper_wave = max_wave + 0.5e-5 # + 0.05 AA

    # For the Na and K lines, I'm going for 2 nm instead.
    # WFC3/UVIS, WFC3/IR, and STIS/IR M/L gratings and grisms reach similar or smaller dispersions,
    # but you always want some binning for SNR.
    if atom+state_str in ("Na_I","K_I"):
        lower_wave = max_wave - 1e-3 # - 1nm
        upper_wave = max_wave + 1e-3 # + 1nm

    upper_index = (np.abs(waves - lower_wave)).argmin()
    lower_index = (np.abs(waves - upper_wave)).argmin()
    print("Getting cross section over {:.0F} indices from wavelengths {:.3F} to {:.3F} AA.".format(np.abs(upper_index - lower_index),
                                                                                                          lower_wave*1e4,
                                                                                                          upper_wave*1e4))
    max_sigma = sigma[lower_index:upper_index]
    max_waves = waves[lower_index:upper_index]
        
    # Define the bandpass in micron
    wmin, wmax = (np.min(max_waves),np.max(max_waves))
    band = wmax - wmin

    # Retrieve the average cross-section on this range
    sec = np.mean(max_sigma)
    wave = np.median(max_waves)
    hw = 0.5*(np.max(max_waves)-np.min(max_waves))

    print("Maximum cross section for {} at index {}, wavenumber {:.0F} cm^-1, wavelength {:.0F} nm.".format(atom+state_str,
                                                                                                            max_index,
                                                                                                            (10*1000)/wave,
                                                                                                            wave*1000))
    absorption_dict[atom+state_str] = [wave,sec,hw,mass]

In [ ]:
# Make binned down absorption dict.
bin_cross_dict = {}
for spec in tqdm(list(full_cross_dict.keys()),
                desc='Binning cross sections to HST resolution...'):
    wave, sigma, mass = full_cross_dict[spec]
    wave = np.array([i for i in list(reversed(wave))])
    sigma = np.array([i for i in list(reversed(sigma))])

    # Get binned down version, where binning elements are
    # 0.1 AA = 0.01 nm at wavelengths < 300 nm,
    # and 2 nm at wavelengths > 300 nm.
    lam_step = 1e-5
    bin_w, bin_sig = [],[]
    lam0 = 0.1
    while lam0 < 1.0:
        ok = (wave>lam0) & (wave<lam0+lam_step)
        bin_w.append(np.mean(wave[ok]))
        bin_sig.append(np.mean(sigma[ok]))
        lam0 += lam_step
        if lam0 > 0.3:
            lam_step = 2e-3

    bin_cross_dict[spec] = [np.array(bin_w),np.array(bin_sig),mass]

In [ ]:
### CONFIRMING I GOT WHAT I NEEDED AND SAVING OUT ###
for key in list(absorption_dict.keys()):
    wave, sig, hw, mass = absorption_dict[key]
    print('{}: peaks at {:.0F}+/-{:.3F} nm with cross-section {:.2E} cm^2'.format(key,1000*wave,1000*hw,sig))

filename = 'cross-sections.npy'
np.save(filename,absorption_dict)
filename = 'cross-sections-full.npy'
np.save(filename,full_cross_dict)
filename = 'cross-sections-bin.npy'
np.save(filename,bin_cross_dict)

This is the end of the section you skip if you didn't want to make the cross-sections from scratch. You may now proceed with running every cell hereafter.

In [ ]:
### LOADING AND PLOTTING ###
filename = 'cross-sections.npy'
absorption_dict = np.load(filename,allow_pickle=True).item()

filename = 'cross-sections-bin.npy'
bin_cross_dict = np.load(filename,allow_pickle=True).item()

try:
    filename = 'cross-sections-full.npy'
    full_cross_dict = np.load(filename,allow_pickle=True).item()
except FileNotFoundError:
    print("Full cross section dictionary .npy file not found:\n Full-resolution cross sections will not be available for this run!")

atoms = ('C','O','Na','S','K')
colors = ('dodgerblue','red','lime','goldenrod','darkorchid')

plt.figure(figsize=(10,5))
for atom, color in zip(atoms,colors):
    lam, sigma, hws, mass = absorption_dict[atom+"_I"]
    plt.errorbar(lam,sigma,xerr=hws,marker='s',markersize=10,ls='none',label=atom+"_I",color=color)

plt.xlabel('wavelength [um]')
plt.ylabel('cross section [cm^2]')
plt.xscale('log')
plt.yscale('log')
plt.legend(loc='upper right')
plt.xlim(0.1,5)
plt.xticks(ticks=[0.1,0.2,0.4,0.8,1.0,2.0,5.0],#,10.0],
           labels=['{:.1F}'.format(i) for i in [0.1,0.2,0.4,0.8,1.0,2.0,5.0]])#,10.0]])
plt.ylim((1e-16,1e-12))
plt.show()
plt.close()

plt.figure(figsize=(10,5))
for atom, color in zip(atoms,colors):
    lam, sigma, mass = bin_cross_dict[atom+"_I"]
    plt.plot(lam,sigma,label=atom+"_I",color=color)

plt.xlabel('wavelength [um]')
plt.ylabel('cross section [cm^2]')
plt.xscale('log')
plt.yscale('log')
plt.legend(loc='upper right')
plt.xlim(0.1,5)
plt.xticks(ticks=[0.1,0.2,0.4,0.8,1.0,2.0,5.0],#,10.0],
           labels=['{:.1F}'.format(i) for i in [0.1,0.2,0.4,0.8,1.0,2.0,5.0]])#,10.0]])
plt.ylim((1e-30,1e-10))
plt.show()
plt.close()

In [ ]:
### CONSTANTS OF LENGTH ###
au      = 1.496e13  # cm
rads    = 6.957e10  # cm
rade    = 6.371e8   # cm

### CONSTANTS OF TIME ###
day     = 86400     # s
year    = 3.1536e7  # s

### CONSTANTS OF MASS ###
masss   = 1.989e33  # g
masse   = 5.9722e27 # g
amu     = 1.66e-24  # g

### OTHER CONSTANTS ###
G       = 6.6743e-8 # dyne cm^2 g^-2
kB      = 1.381e-16 # erg K^-1
pi      = np.pi     # no dims
mu0     = 1/(8*pi)  # g cm^-1 s^-2, cgs version of (1 Gauss)^2 / 2 mu0

### PLANETS WE ARE INTERESTED IN ###
tois = ['GJ 1132 b', 'GJ 367 b', 'GJ 486 b',
        'GJ 9827 b', 'GJ 9827 c', 'HD 23472 d',
        'HD 23472 e', 'HD 23472 f', 'HD 260655 b',
        'HD 63433 d', 'K2-36 b', 'Kepler-102 b',
        'Kepler-102 c', 'Kepler-102 d', 'L 168-9 b',
        'L 98-59 b', 'LHS 1678 b', 'TOI-238 b']

best = ['GJ 367 b',]

### PLOTTING STUFF ###
fontsize_axis = 16
fontsize_tick = 14

In [ ]:
### EQUATIONS ###
def torus_volume(a, H, n=3):
    # Torus volume assuming the torus spans n scale heights and starts at r_inner = a.
    r = n*H/2
    R = a + r
    return (pi*(r**2))*(2*pi*R)

def torus_mass(n, m, V):
    # Torus mass based on isotropic mass distribution.
    return n*m*V

def volc(tau, M):
    # Torus supply rate based on the torus mass and how long it takes to escape from the torus.
    return M/tau

def number_dens(fs, H, sig, n=3):
    # Number density which produces an absorption signature of fs in a torus of n scale heights for absorbers of size sig.
    ds = n*H
    return fs/(sig*ds)

def scale_height(T, m, Omega):
    # Plasma diffusion scale height, used here to approximate the scale radius.
    return np.sqrt((2/3)*kB*T/(m*(Omega**2)))

def get_teq(aoR,Teff):
    # 0-albedo equilibrium temperature, needed for scale height.
    return Teff*((1/4)**(1/4))/(aoR**0.5)

def lifetime(aoR, RA, Omega):
    # Minimum torus reservoir lifespan based on uninhibited magnetospheric convection.
    return np.sqrt(2/3)*((RA/aoR)**2)*(1/Omega)

def vesc(m, r):
    # Escape velocity.
    return np.sqrt(2*G*m/r)

def eta_star(b, r, wind, vinf):
    # Eta*, the wind magnetic confinement parameter, which is dimensionless.
    return ((b**2)*mu0)*(r**2)/(wind*vinf)

def _residuals(ra, eta):
    # Residual function, see below.
    lhs = ra**4 - ra**3
    return (lhs-eta)**2

def get_alfven(eta):
    # Owocki 2009 Alfven radius solver.
    results = minimize(_residuals,np.array([20,]),args=(eta,))
    return results.x[0]

def empirical_mass(r):
    # The Chen & Kipping 2017 / Louie et al. 2018 mass-radius relations. Useful if you don't have a planet mass measurement on hand.
    emp_m = 0.9718*(r**3.58)
    if r > 1.23:
        emp_m = 1.436*(r**1.70)
    return emp_m

def get_tidal_power(p,rp,e):
    # Seligman et al. 2024 tidal power relation, scaled by the unknown quality factor.
    return (3.4*1e27)*(p**-5)*(rp**5)*((e/0.01)**2)

Some assumptions:
I typically take the stellar B to be 500 G. This is my attempt to compromise between the mild B fields of G stars (~100 G) and the extreme B fields of M stars (1 kG or more). For M stars in our sample, consider B to be possibly underestimated. For G stars, consider B to be possibly overestimated. As most of the stars in our sample are K and M stars, erring on the side of slightly-higher-than-G-star seems reasonable.
I also take the wind loss rate to be 10^-14 solar masses per year based on our own solar wind. This is usually an underestimate for earlier types of stars, but we won't encounter too many F- or A-orbiting targets here anyhow.

Rationale for how to pick the planets off of Exoplanet Archive (performed Nov 2024 so numbers may change when you try this):
To do these calculations, you have to have a stellar rotation rate, otherwise you will not be able to get a lifetime. So you need to enable that column, and set it to >0 so that we only get stars for which this is measured. This cuts the available targets down to 890. This is a big cut because not many people measure this value, since you normally don't need to know about it.

You are also going to need a known stellar mass, radius, and effective temperature, for getting the eta* parameter that helps you solve for the Alfven radius. Set all of those >0. Target N should fall to 795. These measurements are common so this cut is small.

You need to know the planet's semimajor axis in order to determine if it is inside the Alfven radius. So set orbit semimajor axis >0. The targets are now down to 617.

Since we are interested in outgassing from volcanically-active terrestrial planets, it won't benefit us to include large radii planets. Set planetary radius to bounds [0.0,1.60]. The number of targets is now 87. This is another big cut, because small planets are hard to find.

You could, at this point, make a few extra cuts if you wanted:

a. You could cut on mass to ensure it is terrestrial (e.g. < 4.0ME), but I use the empirical relation when there's no known mass.

b. You could cut on equilibrium temperature to ensure it's not a ball of lava by instellation alone (e.g. T < 1500 K), but I think it's worth looking at ultra-hot rocks.

c. You could cut on eccentricity > 0 to ensure tidal heating, but I think it's worth acknowledging non-eccentric targets that could be intensely volcanic anyhow, i.e. super young planets.

d. You could cut on whether or not this is the default parameter set. If you do that, it drops us to 41 available targets. I instead prefer to take all options, and then pick and choose between the planets that have duplicate entries.

At this stage, I take my targets. The columns I need to have are:

Whether it has been detected in RVs, which tells us how good the ecc measurement is

Planet parameter reference, again useful for knowing whether the ecc measurement is good

Planet name

Planet orbit semimajor axis

Planet orbit period, needed if you are going to compute tidal heating rates

Planet radius

Planet mass

Planet eccentricity, which may be unknown

Planet equilibrium temperature, which may be unknown

Stellar radius

Stellar mass

Stellar effective temperature

Stellar rotation period

System V magnitude

I attached my table to the GitHub so you can quickly recreate this, but also, you should play around with it, see what else is out there. Some notes on special treatments I made:
1. LHS 1815 b is in my table as the Gan+ 2020 entry, but I used the better mass constraint from Luque+ 2022.
2. KOI 4777.01 has a weird mass measurement of <99 Earth masses on the archive, but their paper says it's meant to be <0.34 Earth masses. I think someone entered something wrong, so I replaced the data entry there with the correct upper bound.
3. Kepler-107 c makes it through all the checks but the mass measurements are really weird, like 2x iron density weird. I toss this target for safety's sake.
4. LHS 1678 b and c have two retrieved entries, Silverstein+ 2022 and 2024. I took the 2024 one since its ecc measurements had bounds, not just upper limits.
4. GJ 367 b retrieved two entries, Goffo+ 2023 and Lam+ 2021. Goffo+ 2023 is the default parameter set so I nabbed it.
5. GJ 486 b retrieved two entries, Trifonov+ 2021 and Caballero+ 2022. It did not collect the most recent set, Weiner Mansfield+ 2024, because that one didn't make a P_rot measurement for the star. I took most of the info from Weiner Mansfield+ 2024, subbing P_rot with Caballero+ 2022 data.
6. The GJ 9827 system had a plethora of measurements come up. Only Bonomo+ 2023 tried to estimate the eccentricity, so I took that one. It's only an upper bound though, so it's not particularly robust. But can't hurt to try.
7. K2-141 b had a Bonomo+ 2023 entry as well so I took that.
8. K2-36 b was the same as above.
9. The L 98-59 system was supplied entirely by Demangeon+ 2024.
10. LHS 1140 c retrieved Ment+ 2019 and Cadieux+ 2024. The latter was taken as it is the default parameter set.
11. For GJ 1132 b, I use the info from Xue+ 2024, except the stellar rotation which I get from Bonfils+ 2018.
12. For HD 93963 A b, I use the info from Polanski+ 2024, except the stellar rotation which I get from Serrano+ 2022.
13. For HIP 29442 b and c, I use the info from Egger+ 2024, except the stellar rotation which I get from Damasso+ 2023.
14. For K2-233 b and c, I use the info from Lillo-Box+ 2020, except the stellar rotation which I get from David+ 2018. Since they are both oddly massive by these studies, I let them go.
15. For the TOI-700 system, I use Gilbert+ 2023 for most parameters, and Gilbert+ 2020 for the stellar rotation.
16. For TRAPPIST-1, I use Agol+ 2021 for most parameters, Gillon+ 2016 for the stellar rotation, and Grimm+ 2018 for the eccentricity.

In [ ]:
# Read in our table.
data = pd.read_excel('plasma_planets3.xlsx',header=96) # you may need to adjust this row if you made a new table!
print(list(data.keys())) # check that we got the header row right, this should say stuff like 'pl_name'

Now let's establish the workflow. We have x number of planets that we need to make some calculations for, as follows:
1. Compute vinf
2. Compute eta*
3. Solve for RA
4. Determine tau
5. Determine tidal heating, where applicable

Then, for each absorbing species and desired absorption strength:
1. Determine number density needed to produce absorption
2. Determine torus steady state mass with that number density
3. Determine torus mass input to sustain that steady state mass

In [ ]:
### VALUES THAT I AM ASSUMING ###
stellar_B = 500 # gauss, built into the mu0 value
wind_mass = (1e-14)*masss/year # g/s

In [ ]:
### BIG WRAPPER FUNCTIONS ###
def process_planet(pdata,k):
    # 0. Some useful numbers
    aor = (au/rads)*(pdata['pl_orbsmax'][k]/pdata['st_rad'][k]) # no dims, term out front is ~215
    if pdata['pl_name'][k] == 'Io':
        aor = 6 # units for Io are a bit funky: R* is supposed to be Jupiter radius
    omega = 2*pi/(day*pdata['st_rotp'][k]) # in 1/s
    Teff=pdata['st_teff'][k] # in K
    teq = get_teq(aoR=aor,Teff=Teff) # also in K, assuming zero albedo
    if pdata['pl_name'][k] == 'Io':
        teq = 110 # Io is orbiting Jupiter which is orbiting the Sun which really sets Teq
    ecc_err = [pdata['pl_orbeccenerr1'][k],pdata['pl_orbeccenerr2'][k]] # eccentricity bounds
    V = pdata['sy_vmag'][k] # apparent Vega V mag

    # 1. Compute vinf
    vinf = vesc(m=masss*pdata['st_mass'][k],
                r=rads*pdata['st_rad'][k]) # in cm/s
    
    # 2. Compute eta*
    eta = eta_star(b=stellar_B,
                   r=rads*pdata['st_rad'][k],
                   wind=wind_mass,
                   vinf=vinf) # no dims
    
    # 3. Solve for RA
    alfven = get_alfven(eta=eta) # in stellar radii
    if pdata['pl_name'][k] == 'Io':
        alfven = 20

    # 4. Determine torus lifetime
    tau = lifetime(aoR=aor,
                   RA=alfven,
                   Omega=omega) # in s
    if pdata['pl_name'][k] == 'Io':
        tau = 50*day # 50 days is chosen since 20 to 80 days is the range of tau measured for Io and (20+80)/2 = 50
    
    # 5. Determine tidal heating, where applicable
    tides = 0 # in erg/s, set to 0 unless all the parameters needed to calculate tides are available
    if all(not np.isnan(x) for x in (pdata['pl_orbper'][k],pdata['pl_rade'][k],pdata['pl_orbeccen'][k])):
        tides = get_tidal_power(pdata['pl_orbper'][k],pdata['pl_rade'][k],pdata['pl_orbeccen'][k])
        if np.isnan(tides):
            tides = 0
        if not np.isfinite(tides):
            tides = 0
    if pdata['pl_name'][k] == 'Io':
        tides = 1.6e21 # from Seligman+ 2024
    
    # Trim any planets that have weird masses
    pmass = pdata['pl_bmasse'][k]
    if (np.isnan(pmass) or pmass == 0):
        pmass = empirical_mass(r=pdata['pl_rade'][k]) # sub in with empirical relation
    if pmass > 8.0:
        print(pdata['pl_name'][k],'had an abnormally huge mass ({} Earth masses with radius {} Earth radii) and was discarded.'.format(pmass,pdata['pl_rade'][k]))
        return False
    else:
        return alfven, tau, tides, aor, teq, V, pdata['pl_rade'][k], pmass, Teff, ecc_err

def process_abs(pdata,k,tau,mol_mass,mol_sigma,
                fs=100e-6,n=3):
    # 0. Some useful numbers
    aor = (au/rads)*(pdata['pl_orbsmax'][k]/pdata['st_rad'][k]) # no dims, term out front is ~215
    if pdata['pl_name'][k] == 'Io':
        aor = 6
    omega = 2*pi/(day*pdata['st_rotp'][k]) # in 1/s
    Teff=pdata['st_teff'][k] # in K
    teq = get_teq(aoR=aor,Teff=Teff) # in K

    # 1. Determine number density needed to produce absorption
    h = scale_height(T=teq,m=amu*mol_mass,Omega=omega) # in cm
    n_dens = number_dens(fs,h,mol_sigma,n) # in cm^-3

    # 2. Determine torus steady state mass with that number density
    V = torus_volume(au*pdata['pl_orbsmax'][k],h,n) # in cm^3
    tMass = torus_mass(n_dens,amu*mol_mass,V) # in g

    # 3. Determine torus mass input to sustain that steady state mass
    mdot = volc(tau,tMass) # in g/s

    # 4. Get column density, out of academic interest
    ncol = n*h*n_dens # in cm^-2
    
    return h, n_dens, tMass, mdot, ncol

In [ ]:
### COMPILING THESE NUMBERS TO MAKE PLOTS WITH ###
alfvens, taus, tideses, aors, teqs, Vs, rps, mps, names, ecc_errs = [], [], [], [], [], [], [], [], [], []
a, P, ecc, mstar, rstar, pstar, Tstar, sys_dist = [], [], [], [], [], [], [], []
for k in range(len(data)):
    try:
        alfven, tau, tides, aor, teq, V, rp, mp, Teff, ecc_err = process_planet(data,k)
        alfvens.append(alfven)
        taus.append(tau)
        tideses.append(tides)
        aors.append(aor)
        teqs.append(teq)
        Vs.append(V)
        rps.append(rp)
        mps.append(mp)
        ecc_errs.append(ecc_err)
        names.append(data['pl_name'][k])

        a.append(data['pl_orbsmax'][k])
        P.append(data['pl_orbper'][k])
        ecc.append(data['pl_orbeccen'][k])
        mstar.append(data['st_mass'][k])
        rstar.append(data['st_rad'][k])
        pstar.append(data['st_rotp'][k])
        Tstar.append(Teff)
        sys_dist.append(data['sy_dist'][k])
    except TypeError:
        # this exception catches when the planet was tossed for having a weird mass
        # by making line 6 attempt to unpack a False
        pass

In [ ]:
### PLOTTING FUNCTIONS ###
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap
cmap = plt.get_cmap('RdYlBu')
spectral_cmap = truncate_colormap(cmap, 0.0, 0.55)

def plot_alfven_aor(names,alfvens,aors,rps,Tstar):
    # Rescale size
    F = 1000
    sizes = F*np.log10(np.square(np.array(rps))+1)
    re_size = F*np.log10(np.square(np.array(1))+1)
    
    # Put the data on
    fig, ax = plt.subplots(figsize=(10,8))

    im = ax.scatter(aors,alfvens,c=Tstar,cmap=spectral_cmap,edgecolors='k',s=sizes,vmin=2500,vmax=6000)
    
    for name, alfven, aor, size in zip(names,alfvens,aors,sizes):
        if (all(i <= 40 for i in (alfven,aor))): # truncate range for plot visibility
            if name in tois:
                # if it's one of the targets of interest, highlight it in blue
                ax.scatter(aor,alfven,s=size,color='None',edgecolor='blue',lw=3)
            elif name == 'Io':
                # give Io a special marker
                ax.scatter(aor,alfven,s=size,color='black',edgecolor='black')
                ax.text(aor,alfven-1,s=name,
                        ha='center',va='center',fontsize=fontsize_tick,
                        color='k',path_effects=[pe.withStroke(linewidth=2.25,
                                                              foreground='white')])
            if name in best:
                # it's GJ 367b, give it an extra mark
                ax.scatter(aor,alfven,s=size,color='blue',marker='x',lw=1)
    
    # Create a legend for high-priority targets and GJ 367b
    im2 = ax.scatter(-1000,-1000,color='None',edgecolor='blue',s=re_size,lw=3,label='High-priority targets')
    im3 = ax.scatter(-1000,-1000,color='blue',marker='x',s=re_size,lw=1,label='GJ 367 b')
    lgnd2 = ax.legend(handles=(im2,im3),loc='upper right',borderpad=1.0,labelspacing=1.5,fontsize=fontsize_tick)

    # Dividing line telling us who is in the Alfven radius
    ax.plot([0,50],[0,50],color='red',ls='--')
    ax.text(6,7,s=r'INSIDE R$_A$',rotation=45,
            color='red',weight='bold',fontsize=fontsize_tick)
    ax.arrow(7.5,10,dx=-3,dy=3,color='red',width=0.25)
    
    # Second legend for plot sizes
    labels = ['0.5RE','1.0RE','1.5RE']
    hands = []
    for l in labels:
        hand = ax.scatter(0,0,color='white',edgecolors='k',label=l)
        hands.append(hand)
    lgnd = ax.legend(handles=hands,loc='lower right',ncols=3,columnspacing=0.5,handlelength=3,borderpad=1.0,fontsize=fontsize_tick)#edgecolor='white')
    lgnd.legend_handles[0]._sizes = [F*np.log10(0.5**2+1)]
    lgnd.legend_handles[1]._sizes = [F*np.log10(1.0**2+1)]
    lgnd.legend_handles[2]._sizes = [F*np.log10(1.5**2+1)]

    ax.add_artist(lgnd2)
    ax.add_artist(lgnd)
    
    # Make it pretty
    plt.rc('grid',color='grey',alpha=0.5,ls=':')
    plt.grid()
    plt.xlim(0,40)
    plt.ylim(5,45)
    plt.xticks(ticks=[0,5,10,15,20,25,30,35,40])
    plt.yticks(ticks=[5,10,15,20,25,30,35,40,45])
    ax.set_xlabel('Semimajor Axis [Stellar Radii]',fontsize=fontsize_axis)
    ax.set_ylabel('Alfven Radius [Stellar Radii]',fontsize=fontsize_axis)
    ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
    cb = plt.colorbar(mappable=im,)
    cb.set_label('Stellar Temperature [K]', fontsize=fontsize_axis)
    cb.ax.tick_params(labelsize=fontsize_tick)

    plt.savefig('alfven_radii.png',dpi=300,bbox_inches='tight')
    plt.close()

def plot_timescale(taus,alfvens,aors,names,rps,Tstar):
    # Rescale size
    F = 1000
    re_size = F*np.log10(np.square(np.array(1))+1)
    sizes = np.array(F*np.log10(np.square(np.array(rps))+1))
    # Put the data on
    fig, ax = plt.subplots(figsize=(10,8))
    lols = np.array([i/j for i,j in zip(aors,alfvens)]) # compute a/RA values
    staus = np.array([t/day for t in taus]) # tau in days
    Tstar = np.array(Tstar) # stellr temperatures
    
    # Only plot those which orbit at a<RA
    im = ax.scatter(lols[lols <= 1],staus[lols <= 1],c=Tstar[lols <= 1],cmap=spectral_cmap,edgecolors='k',s=sizes[lols <= 1],vmin=2500,vmax=6000)
    for name, lol, tau, size in zip(names,lols,staus,sizes):
        if (lol <= 1): # again, only those at a<RA
            if name in tois:
                # if it's one of the targets of interest, highlight it in blue
                ax.scatter(lol,tau,s=size,color='None',edgecolor='blue',lw=3)
            elif name == 'Io':
                # give Io a special marker
                ax.scatter(lol,tau,s=size,color='black',edgecolor='black')
                ax.text(lol,tau-10,s=name,
                        ha='center',va='center',fontsize=fontsize_tick,
                        color='k',path_effects=[pe.withStroke(linewidth=2.25,
                                                              foreground='white')])
            if name in best:
                # it's GJ 367b, give it an extra mark
                ax.scatter(lol,tau,s=size,color='blue',marker='x',lw=1)
    
    # Create a legend for high-priority targets and GJ 367b
    im2 = ax.scatter(1e12,1e12,color='None',edgecolor='blue',s=re_size,lw=3,label='High-priority targets')
    im3 = ax.scatter(1e12,1e12,color='blue',marker='x',s=re_size,lw=1,label='GJ 367 b')
    lgnd2 = ax.legend(handles=(im2,im3),loc='lower left',borderpad=1.0,labelspacing=1.5,fontsize=fontsize_tick)

    # Second legend for plot sizes
    labels = ['0.5RE','1.0RE','1.5RE']
    hands = []
    for l in labels:
        hand = ax.scatter(0,0,color='white',edgecolors='k',label=l)
        hands.append(hand)
    lgnd = ax.legend(handles=hands,loc='upper right',ncols=3,columnspacing=0.5,handlelength=3,borderpad=1.0,fontsize=fontsize_tick)#edgecolor='white')
    lgnd.legend_handles[0]._sizes = [F*np.log10(0.5**2+1)]
    lgnd.legend_handles[1]._sizes = [F*np.log10(1.0**2+1)]
    lgnd.legend_handles[2]._sizes = [F*np.log10(1.5**2+1)]

    ax.add_artist(lgnd2)
    ax.add_artist(lgnd)
    
    # Make it pretty
    plt.rc('grid',color='grey',alpha=0.5,ls=':')
    plt.grid()
    plt.xlim(0,1)
    plt.yscale('log')
    plt.ylim(0.5,500)
    plt.xticks(ticks=[0,0.25,0.5,0.75,1.00])
    plt.yticks(ticks=[1,10,100,500],
               labels=['1.0','10.0','100.0','500.0'])
    ax.set_xlabel('Semimajor Axis [Alfven Radii]',fontsize=fontsize_axis)
    ax.set_ylabel('Magnetospheric Convection Timescale [Days]',fontsize=fontsize_axis)
    ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
    cb = plt.colorbar(mappable=im)
    cb.set_label('Stellar Temperature [K]', fontsize=fontsize_axis)
    cb.ax.tick_params(labelsize=fontsize_tick)

    plt.savefig('alfven_tau.png',dpi=300,bbox_inches='tight')
    plt.close()

def plot_tau_tides(names,taus,masses,tides,alfvens,aors,rps,v):
    zl = 'Magnitude [V]'
    C = 'Greens'
    vmin, vmax = (5,13)
    
    # Rescale size
    F = 1000
    re_size = F*np.log10(np.square(np.array(1))+1)
    # Put the data on
    fig, ax = plt.subplots(figsize=(10,8))
    lols = np.array([i/j for i,j in zip(aors,alfvens)])
    sizes = np.array(F*np.log10(np.square(np.array(rps))+1))[lols <= 1]
    stides = np.array([t/(masse*m) for t,m in zip(tides,masses)])[lols <= 1] # tidal heating per unit mass
    staus = np.array([t/day for t in taus])[lols <= 1]
    v = np.array(v)[lols <= 1]
    snames=np.array(names)[lols <= 1]

    # Prevent the Io point from becoming blue
    io_index = np.argwhere(snames=='Io')[0]
    original_io_size = sizes[io_index]
    sizes[io_index] = 0

    im = ax.scatter(stides[stides > 0],staus[stides > 0],c=v[stides > 0],cmap=C,edgecolors='blue',s=sizes[stides > 0],
                    vmin=vmin,vmax=vmax,lw=3)
    for name, tide, tau, size in zip(snames[stides > 0],stides[stides > 0],staus[stides > 0],sizes[stides > 0]):
        if name == 'Io':
            ax.scatter(tide,tau,s=original_io_size,c=5,cmap=C,edgecolor='black')
            ax.text(tide,tau-15,s=name,
                    ha='center',va='center',fontsize=fontsize_tick,
                    color='k',path_effects=[pe.withStroke(linewidth=2.25,
                                                            foreground='white')])
        if name in best:
            ax.scatter(tide,tau,s=size,color='None',edgecolor='blue',lw=3)
            ax.scatter(tide,tau,s=size,color='blue',marker='x',lw=1)

    # Create a legend for targets
    im2 = ax.scatter(1e12,1e12,color='None',edgecolor='blue',s=re_size,lw=3,label='High-priority targets')
    im3 = ax.scatter(1e12,1e12,color='blue',marker='x',s=re_size,lw=1,label='GJ 367 b')
    lgnd2 = ax.legend(handles=(im2,im3),loc='upper left',borderpad=1.0,labelspacing=1.5,fontsize=fontsize_tick)
            
    # And second legend for plot sizes
    labels = ['0.5RE','1.0RE','1.5RE']
    hands = []
    for l in labels:
        hand = ax.scatter(0,0,color='white',edgecolors='k',label=l)
        hands.append(hand)
    lgnd = ax.legend(handles=hands,loc='lower left',ncols=3,columnspacing=0.5,handlelength=3,borderpad=1.0,fontsize=fontsize_tick)#edgecolor='white')
    lgnd.legend_handles[0]._sizes = [F*np.log10(0.5**2+1)]
    lgnd.legend_handles[1]._sizes = [F*np.log10(1.0**2+1)]
    lgnd.legend_handles[2]._sizes = [F*np.log10(1.5**2+1)]

    ax.add_artist(lgnd2)
    ax.add_artist(lgnd)
    
    # Make it pretty
    plt.rc('grid',color='grey',alpha=0.5,ls=':')
    plt.grid()
    plt.xlim(1e-5,1e4)
    plt.xticks([1e-5,1e-3,1e-1,1e1,1e3])
    plt.xscale('log')
    plt.yscale('log')
    plt.ylim(0.5,100)
    y_ticks = [1,5,10,50,100,500]
    plt.yticks(ticks=y_ticks,
            labels=['%.1f' % x for x in y_ticks])
    plt.rcParams['mathtext.default'] = 'regular' # prevent E from getting italicized
    ax.set_xlabel(r'$\dot{E}_{\rm heat}$/Im(k$_2$)/Mass [erg/s/g]',fontsize=fontsize_axis)
    ax.set_ylabel('Magnetospheric Convection Timescale [Days]',fontsize=fontsize_axis)
    cb = plt.colorbar(mappable=im)
    cb.set_label(zl, fontsize=fontsize_axis)
    cb.ax.tick_params(labelsize=fontsize_tick)
    ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
    plt.savefig('alfven_tides.png',dpi=300,bbox_inches='tight')
    plt.close()

def generate_table_lifetimes(names,a,ra,P,ecc,rp,teq,mstar,rstar,pstar,vmag,tides,taus,ecc_errs,dists):
    # Sort everything by torus lifetime.
    b = [k for k in zip(names,a,ra,P,ecc,rp,teq,mstar,rstar,pstar,vmag,tides,ecc_errs,dists,taus) if not np.isnan(k[-1])]
    b = list(reversed(sorted(b,key=lambda x:x[-1])))
    names,a,ra,P,ecc,rp,teq,mstar,rstar,pstar,vmag,tides,ecc_errs,dists,taus = [[x[i] for x in b] for i in range(len(b[0]))]

    # Convert alfven radius to AU.
    ra = [x*rs*rads/au for x,rs in zip(ra,rstar)]
    # Convert timescale to days.
    taus = [x/day for x in taus]
    # Get H for 32 amu.
    H = []
    for i in range(len(names)):
        H.append(scale_height(teq[i],32*amu,2*pi/(pstar[i]*day)))

    # Fill out table.tex file.
    with open('table.tex',mode='w') as f:
        for i in range(len(names)):
            if names[i] == "Io":
                continue
            if (tides[i] > 0 and a[i]/ra[i] <= 1):
                if np.isnan(ecc_errs[i][0]):
                    # condition for those that only have an ecc upper bound
                    f.write('{} & {:.3F} & {:.3F} & {:.2F} & {:.3F} & $<${:.3F} & {:.3F} & {:.0F} & {:.2F} & {:.2F} & {:.2F} & {:.3F} & {:.0F} & {:.2E} & {:.0F} & \\\\ \n'.format(names[i],
                                            a[i],
                                            ra[i],
                                            H[i]/(100*1000*1e5), # turn cm into 10^5 km
                                            P[i],
                                            ecc[i],
                                            rp[i],
                                            teq[i],
                                            mstar[i],
                                            rstar[i],
                                            pstar[i],
                                            vmag[i],
                                            dists[i],
                                            tides[i],
                                            taus[i],))
                else:
                    # ecc constrained
                    f.write('{} & {:.3F} & {:.3F} & {:.2F} & {:.3F} & {:.3F}$^{{{:.3F}}}_{{{:.3f}}}$ & {:.3F} & {:.0F} & {:.2F} & {:.2F} & {:.2F} & {:.3F} & {:.0F} & {:.2E} & {:.0F} & \\\\ \n'.format(names[i],
                                            a[i],
                                            ra[i],
                                            H[i]/(100*1000*1e5), # turn cm into 10^5 km
                                            P[i],
                                            ecc[i],
                                            ecc_errs[i][0],
                                            ecc_errs[i][1],
                                            rp[i],
                                            teq[i],
                                            mstar[i],
                                            rstar[i],
                                            pstar[i],
                                            vmag[i],
                                            dists[i],
                                            tides[i],
                                            taus[i],))
                
def generate_table_outgassing(names,mdots,taus,tmasses):
    # Sort everything by torus lifetime.
    b = [k for k in zip(names,mdots,tmasses,taus)]
    b = list(reversed(sorted(b,key=lambda x:x[-1])))
    names,mdots,tmasses,taus = [[x[i] for x in b] for i in range(len(b[0]))]
    
    # Find total outgassing rate of atoms and molecules in ton/s.
    total_atoms = [np.sum(m)/1e6 for m in mdots]

    # Get quasi-steady state mass in Mton.
    torus_mass = [np.sum(m)/1e12 for m in tmasses]
    

    with open('table2.tex',mode='w') as f:
        for i in range(len(names)):
            if names[i] == "Io":
                continue
            # A bit of aesthetic rounding
            # Also shouldn't bother with too many decimals given our uncertainty
            mdot_planet = [k/1e6 for k in mdots[i]]
            mdot_planet = ["{:.0F}".format(f) if f>10 else "{:.1F}".format(f) for f in mdot_planet]
            if total_atoms[i] > 10:
                total_atoms_planet = int(total_atoms[i])
            else:
                total_atoms_planet = np.round(total_atoms[i],decimals=1)
            if torus_mass[i] > 10:
                torus_mass_planet = int(torus_mass[i])
            else:
                torus_mass_planet = np.round(torus_mass[i],decimals=1)
            f.write('{} & {} & {} & {} & {} & {} & {} & {} \\\\ \n'.format(names[i],
                                                                                                mdot_planet[0],
                                                                                                mdot_planet[1],
                                                                                                mdot_planet[2],
                                                                                                mdot_planet[3],
                                                                                                mdot_planet[4],
                                                                                                total_atoms_planet,
                                                                                                torus_mass_planet))

In [ ]:
# Generate all plots and make the table.
plot_alfven_aor(names,alfvens,aors,rps,Tstar)
plot_timescale(taus,alfvens,aors,names,rps,Tstar)
plot_tau_tides(names,taus,mps,tideses,alfvens,aors,rps,Vs)
generate_table_lifetimes(names,a,alfvens,P,ecc,rps,teqs,mstar,rstar,pstar,Vs,tideses,taus,ecc_errs,sys_dist)

In [ ]:
# Use the absorption dict, which contains [central lambda,cross-section,halfwidth,mass], to get n_dens needed for 10% absorption
hs, n_denses, tMasses, mdots, ncols = [],[],[],[],[]
new_names = []
new_taus = []
for k in range(len(data)):
    try:
        alfven, tau, tides, aor, teq, V, rp, mp, Teff, ecc_err = process_planet(data,k)
        if alfven < aor:
            print(1/0)
        if tides == 0:
            print(1/0)
        hk, nk, tk, mk, nck = [],[],[],[],[]
        for spec in list(absorption_dict.keys()):
            central_lams,mol_sigma,halfwidths,mol_mass = absorption_dict[spec]
            
            F = 0.10 # 10% absorption
            
            h_i, n_dens_i, tMass_i, mdot_i, ncol_i = process_abs(data,k,tau,mol_mass,mol_sigma,
                                                fs=F,n=3)
            
            hk.append(h_i)
            nk.append(n_dens_i)
            tk.append(tMass_i)
            mk.append(mdot_i)
            nck.append(ncol_i)
        hs.append(hk)
        n_denses.append(nk)
        tMasses.append(tk)
        mdots.append(mk)
        ncols.append(nck)
        new_names.append(data['pl_name'][k])
        new_taus.append(tau)
    except:
        pass
generate_table_outgassing(new_names,mdots,new_taus,tMasses)

Lastly, we need a plot of the stellar spectrum of GJ 367 with and without torus contamination.

In [ ]:
try:
    print(list(full_cross_dict.keys())[0])
except:
    print("Oops, this cell does not run when you don't have the full cross-section dict.")
    print(1/0) # deliberately cause an exception, sorry I know that's rude of me

M, Teff, logg = (-0.01, 3522, 4.77) # Goffo+ 2023 stellar parameters

# Use pysynphot to get the phoenix model for GJ 367 A
sp = S.Icat('phoenix', Teff, M, logg)
# angstrom to micron
sp.convert('micron')
plt.figure(figsize=(20,5))
plt.plot(sp.wave,sp.flux)
plt.xlabel(sp.waveunits)
plt.ylabel(sp.fluxunits)
plt.xlim(0.1,1.00)
plt.show()
plt.close()

# Truncate flux to our range of interest and normalize it
flux = sp.flux
wave = sp.wave
ok = (wave>0.1) & (wave<10)
fluxGJ = flux[ok]/np.max(flux[ok])
waveGJ = wave[ok]

# For each species, need to get its scale height
# And we need to interpolate the PHOENIX model onto this grid
from scipy.interpolate import interp1d
interpolated = False
full_cross_dict_H = {}
bin_cross_dict_H = {}
plt.figure(figsize=(20,5))
colors = ('dodgerblue','red','lime','goldenrod','darkorchid')
colors_dict = {}
for spec, color in zip(list(full_cross_dict.keys()),colors):
    colors_dict[spec] = color
for spec in tqdm(list(full_cross_dict.keys()),
                desc='Getting scale heights and adjusting resolution of spectrum to match Cthulhu...'):
    wave, sigma, mass = full_cross_dict[spec]
    wave = np.array([i for i in list(reversed(wave))])
    sigma = np.array([i for i in list(reversed(sigma))])

    h = scale_height(T=Teff,m=mass*amu,Omega=2*pi/(day*51.3))

    full_cross_dict_H[spec] = [wave,sigma,h]
    
    if not interpolated:
        interpolater = interp1d(waveGJ,fluxGJ,fill_value=0,bounds_error=False)
        new_flux = interpolater(wave)
        waveGJ = wave
        fluxGJ = new_flux
        interpolated = True
    
    wave, sigma, mass = bin_cross_dict[spec]
    bin_cross_dict_H[spec] = [wave,sigma,h]

    plt.plot(wave,sigma,label=spec,color=colors_dict[spec])
plt.legend()
plt.xlabel('wavelength [um]')
plt.ylabel('cross-section [cm2]')
plt.yscale('log')
plt.xlim(0.10, 1.00)
plt.ylim(1e-25,1e-10)
plt.show()
plt.close()

# Recall that f = Ncol ds = n sigma 3 H
# We have h for each species and sigma for each wavelength for each species
# n was determined in the cell above, let's go get it
for n_dens, name in zip(n_denses, new_names):
    if name == 'GJ 367 b':
        number_densities = n_dens
ndens_dict = {}
for i, key in enumerate(list(absorption_dict.keys())):
    ndens_dict[key] = number_densities[i]

# Now that we have gotten n for each species, we can get f.
modified_flux = np.copy(fluxGJ)
mod_fluxes = []
plt.figure(figsize=(10,5))
for spec in tqdm(list(absorption_dict.keys()),
                desc='Computing absorption...'):
    # First, get the high resolution modified spectrum for the top panel
    wave, sigma, h = full_cross_dict_H[spec]
    n_dens = ndens_dict[spec]
    f = 1 - (n_dens * sigma * 3 * h)
    f[f<0] = 0
    mod_fluxes.append(modified_flux*f)
    
    # Then get the low resolution absorption lines for the bottom panel
    wave, sigma, h = bin_cross_dict_H[spec]
    bin_f = (n_dens * sigma * 3 * h)
    bin_f[bin_f<0] = 0

    plt.plot(wave,100*bin_f,label=spec,color=colors_dict[spec]) # plot the absorption of each species by wavelength
plt.legend()
plt.xlabel('wavelength [um]')
plt.ylabel('absorption [%]')
plt.xscale('log')
plt.xlim(0.10, 1.00)
plt.ylim(0.0,15.0)
plt.show()
plt.close()

# Let's look at the high res spectrum and make sure it makes sense
fig, ax = plt.subplots(figsize=(20,5))
for mod_flux,spec in zip(mod_fluxes,list(full_cross_dict.keys())):
    ax.plot(waveGJ,mod_flux,ls='-',color=colors_dict[spec],label=str.replace(spec,"_I",""))
ax.plot(waveGJ,fluxGJ,ls='-',color='k',alpha=0.75)
ax.legend(loc='lower right',fontsize=fontsize_tick)

# Making the plot pretty
ax.set_yscale('log')
ax.set_ylim(1e-17,10)
yt = [1e-16,1e-12,1e-8,1e-4,1]
ax.set_yticks(yt)
ax.set_yticklabels(['{:.0E}'.format(f) for f in yt])

ax.set_xscale('log')
ax.set_xlim(0.1,1.0)
xt = [0.1,0.2,0.3,0.4,0.5,0.75,1.00]
ax.set_xticks(xt)
ax.set_xticklabels(['{:.2F}'.format(f) for f in xt])

from matplotlib.ticker import NullFormatter as nf
ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
ax.xaxis.set_minor_formatter(nf())
ax.set_xlabel(r'Wavelength [$\mu$m]',fontsize=fontsize_axis)
ax.set_ylabel('Flux [arbitrary units]',fontsize=fontsize_axis)
plt.rc('grid',color='grey',alpha=0.5,ls=':')
plt.grid()

plt.show()
plt.close()

In [ ]:
# Now let's make the two-panel plot!
from matplotlib.ticker import NullFormatter as nf
colors = ('dodgerblue','red','lime','goldenrod','darkorchid')
colors_dict = {}
for spec, color in zip(('C','O','Na','S','K'),colors):
    colors_dict[spec] = color

absorptions = {}
fig, ax = plt.subplots(nrows=2,ncols=1,figsize=(20,10),sharex=True)
plt.subplots_adjust(hspace=0.05)
for mod_flux,spec in zip(mod_fluxes,list(full_cross_dict.keys())):
    ax[0].plot(waveGJ,mod_flux,ls='-',color=colors_dict[str.replace(spec,"_I","")],label=str.replace(spec,"_I",""))
ax[0].plot(waveGJ,fluxGJ,ls='-',color='k',alpha=0.75)
ax[0].legend(loc='lower center',fontsize=fontsize_tick, ncols=5)

# Making the plot pretty
ax[0].set_yscale('log')
ax[0].set_ylim(1e-20,10)
yt = [1e-20,1e-15,1e-10,1e-5,1]
ax[0].set_yticks(yt)
ax[0].set_yticklabels([r'$10^{-20}$',r'$10^{-15}$',r'$10^{-10}$',r'$10^{-5}$','1'])

ax[0].set_xscale('log')
ax[0].set_xlim(0.1,1.0)
xt = [0.1,0.2,0.3,0.4,0.5,0.75,1.00]
ax[0].set_xticks(xt)
ax[0].set_xticklabels([''.format(f) for f in xt])

from matplotlib.ticker import NullFormatter as nf
ax[0].tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
ax[0].xaxis.set_minor_formatter(nf())
ax[0].set_xlabel('',fontsize=fontsize_axis)
ax[0].set_ylabel('Flux [arbitrary units]',fontsize=fontsize_axis)
plt.rc('grid',color='grey',alpha=0.5,ls=':')
ax[0].grid()

hands = []
for spec in tqdm(list(absorption_dict.keys()),
                desc='Computing absorption...'):
    if spec in ("Na_II","Na_III","K_II","K_III","C_III","O_II","O_III"):
        continue
    wave, sigma, h = bin_cross_dict_H[spec]
    ok = (wave>0.1) & (wave<1.0)
    n_dens = ndens_dict[spec]
    f = (n_dens * sigma * 3 * h)
    f[f<0] = 0
    f[f>1] = 1
    print('column density of {}: {:.2E} cm-2'.format(spec,n_dens * 3 * h))
    absorptions[spec] = 1-f

    spec = str.replace(spec,"_",'')
    spec = str.replace(spec,"I",'')
    hand, = ax[1].plot(wave,100*(1-f),label=spec,color=colors_dict[spec]) # plot the absorption of each species by wavelength
    hands.append(hand)

legend1 = ax[1].legend(handles=hands,ncols=5,loc='lower center',fontsize=fontsize_tick)
ax[1].add_artist(legend1)

ax[1].set_xlabel('Wavelength [nm]',fontsize=fontsize_axis)
ax[1].set_ylabel('Observed/Expected Flux [%]',fontsize=fontsize_axis)
ax[1].set_yticks([100,95,90,85])
ax[1].set_xscale('log')
ax[1].set_xlim(0.10, 1.00)
ax[1].set_ylim(80,105)
ax[1].set_xticks([0.1,0.25,0.5,0.75,1.0])
ax[1].set_xticklabels(labels=['100','250','500','750','1000'])
ax[1].tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)
ax[1].xaxis.set_minor_formatter(nf())
plt.rc('grid',color='grey',alpha=0.5,ls=':')
ax[1].grid(zorder=1)
plt.savefig('gj_367.png',dpi=300,bbox_inches='tight')
plt.show()
plt.close()

And you're done. :)

BONUS: Everything below is for generating the appendix figures.

In [ ]:
### MORE EQUATIONS ###
def instel(Teff, aoR):
    return ((Teff/5772)**4)*((aoR/215.032)**-2)

def lumin(Teff, Rs):
    return (Rs**2)*((Teff/5772)**4)

def shore(v):
    return 1e-3*(v**4)

def xuv(I,L):
    return I*(L**-0.6)

In [ ]:
### PLOTTING FUNC ###
def plot_thermal_shoreline(names,vescs,instels,rps,tides,Tstar):
    # Establish the template and draw the escape lines.
    x = np.linspace(1,100,1000)
    y = np.linspace(1,10000,1000)
    xs = [6,8,10,15,20,25]
    ys = [0.1,1.0,10.0,100.0,1000.0,10000.0]

    l,h = (20,10)
    fig, ax = plt.subplots(figsize=(l,h))
    theta = 0.335*np.arctan(l/h)*180/np.pi

    shoreline = shore(x)

    ax.plot(x,shoreline,color='red',lw=3,zorder=1)
    ax.text(6.9,2.5,s='VOLATILE-DEPLETED',rotation=theta,
            color='red',weight='bold',fontsize=fontsize_tick)

    # Now start putting in the planets.
    F = 3000
    sizes = np.array(F*np.log10(np.square(np.array(rps))+1))
    re_size = 0.75*F*np.log10(np.square(np.array(1))+1)
    im = ax.scatter(vescs,instels,s=sizes,c=Tstar,cmap=spectral_cmap,edgecolor='blue',lw=3,vmin=2500,vmax=6000)

    im2 = ax.scatter(1e12,1e12,color='None',edgecolor='blue',s=re_size,lw=3,label='High-priority targets')
    im3 = ax.scatter(1e12,1e12,color='blue',marker='x',s=re_size,lw=1,label='GJ 367 b')
    lgnd2 = ax.legend(handles=(im2,im3),loc='lower right',borderpad=1.0,labelspacing=3.0,fontsize=fontsize_tick)

    # Label them.
    for k in range(len(names)):
        if names[k] in best:
            ax.scatter(y=instels[k],x=vescs[k],s=sizes[k],color='blue',marker='x',lw=1)
            ax.scatter(y=instels[k],x=vescs[k],s=sizes[k],color='None',edgecolor='blue',lw=3)
    
        
    # Legend for plot sizes
    labels = ['0.5RE','1.0RE','1.5RE']
    hands = []
    for l in labels:
        hand = ax.scatter(0,1e10,color='white',edgecolors='k',label=l)
        hands.append(hand)
    lgnd = ax.legend(handles=hands,loc='upper center',ncols=3,columnspacing=0.5,handlelength=3,borderpad=2.0,fontsize=fontsize_tick)#edgecolor='white')
    lgnd.legend_handles[0]._sizes = [F*np.log10(0.5**2+1)]
    lgnd.legend_handles[1]._sizes = [F*np.log10(1.0**2+1)]
    lgnd.legend_handles[2]._sizes = [F*np.log10(1.5**2+1)]

    ax.add_artist(lgnd2)
    ax.add_artist(lgnd)
    
    # Make it pretty
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xticks(xs)
    ax.set_xticklabels([str(i) for i in xs])
    ax.set_xlabel('Escape Velocity [km/s]',fontsize=fontsize_axis)    
    ax.set_xlim(6,25)
    ax.set_yticks(ys)
    ax.set_yticklabels([str(i) for i in ys])
    ax.set_ylabel('Instellation [w.r.t. Earth]',fontsize=fontsize_axis)
    for yi in ys:
        ax.axhline(y=yi,color='whitesmoke',zorder=0)
    for xi in xs:
        ax.axvline(x=xi,color='whitesmoke',zorder=0)
    ax.tick_params(axis='both',which='both',direction='in',labelsize=fontsize_tick)
    ax.set_ylim(1,10000)
    cb = plt.colorbar(mappable=im)
    cb.set_label('Stellar Temperature [K]', fontsize=fontsize_axis)
    cb.ax.tick_params(labelsize=fontsize_tick)
    ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)

    plt.savefig('thermal_shore.png',dpi=300,bbox_inches='tight')
    plt.close()

In [ ]:
### PLOTTING FUNC ###
def plot_xuv_shoreline(names,vescs,instels,rps,tides,Tstar,rstar):
    # Establish the template and draw the escape lines.
    x = np.linspace(1,100,1000)
    y = np.linspace(1,10000,1000)
    xs = [6,8,10,15,20,25]
    ys = [0.1,1.0,10.0,100.0,1000.0,10000.0]

    l,h = (20,10)
    fig, ax = plt.subplots(figsize=(l,h))
    theta = 0.335*np.arctan(l/h)*180/np.pi

    shoreline = shore(x)

    ax.plot(x,shoreline,color='red',lw=3,zorder=1)
    ax.text(6.9,2.5,s='VOLATILE-DEPLETED',rotation=theta,
            color='red',weight='bold',fontsize=fontsize_tick)

    # Now start putting in the planets.
    F = 3000
    sizes = np.array(F*np.log10(np.square(np.array(rps))+1))
    re_size = 0.75*F*np.log10(np.square(np.array(1))+1)
    # Replace instels with XUV
    lumins = lumin(Tstar,rstar)
    instels = xuv(instels,lumins)
    im = ax.scatter(vescs,instels,s=sizes,c=Tstar,cmap=spectral_cmap,edgecolor='blue',lw=3,vmin=2500,vmax=6000)

    im2 = ax.scatter(1e12,1e12,color='None',edgecolor='blue',s=re_size,lw=3,label='High-priority targets')
    im3 = ax.scatter(1e12,1e12,color='blue',marker='x',s=re_size,lw=1,label='GJ 367 b')
    lgnd2 = ax.legend(handles=(im2,im3),loc='lower right',borderpad=1.0,labelspacing=3.0,fontsize=fontsize_tick)

    # Label them.
    for k in range(len(names)):
        if names[k] in best:
            ax.scatter(y=instels[k],x=vescs[k],s=sizes[k],color='blue',marker='x',lw=1)
            ax.scatter(y=instels[k],x=vescs[k],s=sizes[k],color='None',edgecolor='blue',lw=3)
    
        
    # Legend for plot sizes
    labels = ['0.5RE','1.0RE','1.5RE']
    hands = []
    for l in labels:
        hand = ax.scatter(0,1e10,color='white',edgecolors='k',label=l)
        hands.append(hand)
    lgnd = ax.legend(handles=hands,loc='lower center',ncols=3,columnspacing=0.5,handlelength=3,borderpad=2.0,fontsize=fontsize_tick)#edgecolor='white')
    lgnd.legend_handles[0]._sizes = [F*np.log10(0.5**2+1)]
    lgnd.legend_handles[1]._sizes = [F*np.log10(1.0**2+1)]
    lgnd.legend_handles[2]._sizes = [F*np.log10(1.5**2+1)]

    ax.add_artist(lgnd2)
    ax.add_artist(lgnd)
    
    # Make it pretty
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xticks(xs)
    ax.set_xticklabels([str(i) for i in xs])
    ax.set_xlabel('Escape Velocity [km/s]',fontsize=fontsize_axis)    
    ax.set_xlim(6,25)
    ax.set_yticks(ys)
    ax.set_yticklabels([str(i) for i in ys])
    ax.set_ylabel('Cumulative XUV Flux [w.r.t Earth]',fontsize=fontsize_axis)
    for yi in ys:
        ax.axhline(y=yi,color='whitesmoke',zorder=0)
    for xi in xs:
        ax.axvline(x=xi,color='whitesmoke',zorder=0)
    ax.tick_params(axis='both',which='both',direction='in',labelsize=fontsize_tick)
    ax.set_ylim(1,10000)
    cb = plt.colorbar(mappable=im)
    cb.set_label('Stellar Temperature [K]', fontsize=fontsize_axis)
    cb.ax.tick_params(labelsize=fontsize_tick)
    ax.tick_params(which='both',axis='both',direction='in',labelsize=fontsize_tick)

    plt.savefig('xuv_shore.png',dpi=300,bbox_inches='tight')
    plt.close()

In [ ]:
new_names,vescs,instels,new_rps,check_tides,Tstar,rstar,eccs,aors,alfvens,periods = [],[],[],[],[],[],[],[],[],[],[]
for k in range(len(data)):
    try:
        if data['pl_name'][k] == "Io":
            continue
        alfven, tau, tides, aor, teq, V, rp, mp, Teff, ecc_err = process_planet(data,k)
        rs = data['st_rad'][k]
        ecc = data['pl_orbeccen'][k]
        period = data['pl_orbper'][k]
        v = vesc(m=mp*masse,r=rp*rade)/(100*1000)
        i = instel(Teff,aor)
        if alfven < aor:
            print(1/0)
        if tides != 0:
            new_names.append(data['pl_name'][k])
        else:
            print(1/0)
        vescs.append(v)
        instels.append(i)
        check_tides.append(tides)
        new_rps.append(rp)
        Tstar.append(Teff)
        rstar.append(rs)
        eccs.append(ecc)
        aors.append(aor)
        alfvens.append(alfven)
        periods.append(period*day)
    except:
        pass
plot_thermal_shoreline(new_names,np.array(vescs),np.array(instels),np.array(new_rps),np.array(check_tides),np.array(Tstar))
plot_xuv_shoreline(new_names,np.array(vescs),np.array(instels),np.array(new_rps),np.array(check_tides),np.array(Tstar),np.array(rstar))